#https://www.depends-on-the-definition.com/guide-to-multi-label-classification-with-neural-networks/

$$softmx(z)= P(c_j|z_i) = \frac{e^{z_i}}{\sum{e^{z_i}}}$$
<p>softmax models a probability of z_i being in a class c_j as a multinomial distribution. For only 1 class. The softmax is an approximation
to argmax. There are some problems with softmax as an approximator to argmax. softmax is not scale invariant, smaller
values yield different probabilities, softmax[.1,.2.3,.4] is different than softmax[1,2,3,4]. The effect is still
the same we want the one which is the most probable to have the highest value and the resulting probabilites should
sum to 1. </p>
<p>sigmoid models z as a bernoulli distribution. Use a binary CE loss w/sigmoid for a multiclass multilabel. We want [0,.5,.5] as an output to represent the z_i is a member of classes 2 and 3. </p>
$$sigmoid(z) = \frac{1}{1+e^{-z_i}}$$
<p>Cross entropy</p>

In [11]:
import numpy as np

def softmax(z):
    return np.exp(z)/np.sum(np.exp(z))
#4 is highest value. Even if we normalize it is still the largest value
print(softmax([1.0, 2.0, 3.0, 4.0]))
def sigmoid(z):
    return [1/(1+np.exp(-x)) for x in z]
#sigmoid prob does not add up to 1! Use for multilabel multiclass if paired w/BCE and multiple outputs
print(sigmoid([1,2,3,4]))


[0.0320586  0.08714432 0.23688282 0.64391426]
[0.7310585786300049, 0.8807970779778823, 0.9525741268224334, 0.9820137900379085]


In [12]:
#keras ML/MC example. 
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [15]:
class SmallerVGGNet:
	@staticmethod
	def build(width, height, depth, classes, finalAct="softmax"):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
 
		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1
        # CONV => RELU => POOL
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(3, 3)))
		model.add(Dropout(0.25))
        # (CONV => RELU) * 2 => POOL
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
		# (CONV => RELU) * 2 => POOL
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
        # first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(1024))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		# use a *softmax* activation for single-label classification
		# and *sigmoid* activation for multi-label classification
		model.add(Dense(classes))
		model.add(Activation(finalAct))
		# return the constructed network architecture
		return model

In [19]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
#from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os


In [31]:
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset",default="/Users/dougchang/TDS/keras/keras-multi-label/dataset", required=True,help="path to input dataset directory of images")
ap.add_argument("-m", "--model", default='/Users/dougchang/TDS/keras/keras-multi-label',required=True,help="path to output model")
ap.add_argument("-l", "--labelbin",default="/Users/dougchang/TDS/keras/keras-multi-label", required=True,help="path to output label binarizer")
ap.add_argument("-p", "--plot", type=str, default="plot.png",help="path to output accuracy/loss plot")
args = vars(ap.parse_args())

#need to add the 4 constants to args, test... 

# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)
# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(args["dataset"])))
random.seed(42)
random.shuffle(imagePaths)
 
# initialize the data and labels
data = []
labels = []


[INFO] loading images...


NameError: name 'args' is not defined

In [ ]:
# loop over the input images
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)
 
	# extract set of class labels from the image path and update the
	# labels list
	l = label = imagePath.split(os.path.sep)[-2].split("_")
	labels.append(l)

In [27]:
# load the image
image = cv2.imread(args["image"])
output = imutils.resize(image, width=400)
 
# pre-process the image for classification
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)


NameError: name 'args' is not defined